In [1]:
import random
import gmpy2
import math
import time
import os
import hashlib
import numpy
from tabulate import tabulate

from RSA import RSA
from RSA_OAEP import RSA_OAEP
from MillerRabin import miller_rabin

### Test Miller Rabin

In [2]:
for _ in range(1000):
    n = random.randint(100, 100000)
    assert gmpy2.is_strong_prp(n, 7) == miller_rabin(n, 7), n

### Test RSA

In [3]:
for i in range(100):
    if i % 10 == 0:
        print(i)
    rsa = RSA(random.choice([256, 320, 512]))
    
    m = random.randint(2, 2**100)
    pbk, sk = rsa.gen_key()
    c = rsa.encrypt(m, pbk)

    assert m == rsa.decrypt(c, sk), "RSA not working! :C)"

0
10
20
30
40
50
60
70
80
90


### Test RSA-OAEP

In [5]:
for i in range(100):
    if i % 10 == 0:
        print(i)
    n_bits = random.choice([256, 320, 512])
    m_bits = random.choice([64, 128])
    rsa = RSA_OAEP(n_bits, m=m_bits)
    
    m = os.urandom(m_bits//8)
    pbk, sk = rsa.gen_key()
    c = rsa.encrypt_bytes(m, pbk)
    assert m == rsa.decrypt_bytes(c, sk), "RSA-OAEP not working! :C)"

0
10
20
30
40
50
60
70
80
90


### Time tests

In [6]:
m_arr = [64, 128, 256, 512, 1024, 1536]
res_table = [["pq bits/m bits", *[f"{b}bits" for b in m_arr]]]

for prime_bits in [64, 128, 256, 512, 1024, 1536]:
    res = [f"{prime_bits}bits"]
    for m_bits in m_arr:
        m = os.urandom(m_bits//8)
        start = time.time()
        rsa = RSA(prime_bits)

        pbk, sk = rsa.gen_key()
        try:
            c = rsa.encrypt_bytes(m, pbk)
        except AssertionError:
            res.append("-")
            continue
        assert m == rsa.decrypt_bytes(c, sk)
        res.append(f"{(time.time()-start):.2f}")
    res_table.append(res)

In [7]:
print(tabulate(res_table, tablefmt="github"))

|----------------|--------|---------|---------|---------|----------|----------|
| pq bits/m bits | 64bits | 128bits | 256bits | 512bits | 1024bits | 1536bits |
| 64bits         | 0.00   | 0.00    | -       | -       | -        | -        |
| 128bits        | 0.01   | 0.01    | -       | -       | -        | -        |
| 256bits        | 0.03   | 0.03    | 0.01    | 0.03    | -        | -        |
| 512bits        | 0.44   | 0.23    | 0.65    | 0.08    | -        | -        |
| 1024bits       | 0.52   | 2.52    | 8.72    | 5.31    | 2.48     | 3.60     |
| 1536bits       | 10.55  | 6.35    | 7.36    | 12.89   | 23.96    | 21.15    |


In [9]:
m_arr = [64, 128, 256, 512, 1024, 1536]
res_table = [["pq bits/m bits", *[f"{b}bits" for b in m_arr]]]

for prime_bits in [64, 128, 256, 512, 1024, 1536]:
    res = [f"{prime_bits}bits"]
    for m_bits in m_arr:
        m = os.urandom(m_bits//8)
        start = time.time()
        rsa = RSA_OAEP(prime_bits, m=m_bits)

        pbk, sk = rsa.gen_key()
        try:
            c = rsa.encrypt_bytes(m, pbk)
        except AssertionError:
            res.append("-")
            continue
        assert m == rsa.decrypt_bytes(c, sk)
        res.append(f"{(time.time()-start):.2f}")
    res_table.append(res)

In [10]:
print(tabulate(res_table, tablefmt="github"))

|----------------|--------|---------|---------|---------|----------|----------|
| pq bits/m bits | 64bits | 128bits | 256bits | 512bits | 1024bits | 1536bits |
| 64bits         | -      | -       | -       | -       | -        | -        |
| 128bits        | -      | -       | -       | -       | -        | -        |
| 256bits        | 0.03   | 0.01    | 0.01    | -       | -        | -        |
| 512bits        | 0.08   | 0.63    | 0.36    | 0.43    | -        | -        |
| 1024bits       | 2.90   | 6.95    | 1.93    | 1.81    | 2.22     | 2.29     |
| 1536bits       | 6.11   | 9.99    | 15.14   | 20.38   | 18.73    | 11.06    |
